# Red Neuronal

Este código lee las imágenes estraídas de los códigos anteriores, los transforma y realiza el preproceso de las variables. A continuación, define los distintos modelos estudiados y los evalúa

## Imports

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from pathlib import Path
from tqdm import tqdm
import json

## Paso 0: Definición de parámetros y funciones

In [ ]:
# Estructura de directorios
DATA_DIR = Path.cwd().resolve() / "data"
SENTINEL_DATA_DIR = DATA_DIR / "sentinel"
LANDSAT_DATA_DIR = DATA_DIR / "landsat"

In [ ]:
# Tamaño de las imágenes a las que se han de ajustar los datos extraídos, ya sea truncando o añadiendo píxeles dummies.
# Las imágenes extraídas varían su número de filas y columnas aún siendo de la misma resolución
SHAPE = (3, 42, 42) # Corresponde a un detalle de 420*420 m

In [ ]:
# Decodificaciones para los datos de Landsat
WATER_BIT_FILTER = np.vectorize(lambda num: int(format(num,'#018b')[-8]))
CLEAR_BIT_FILTER = np.vectorize(lambda num: int(format(num,'#018b')[-7]))

In [ ]:
def pxl20to10(arr20, shape):
    '''
    Fragmenta una matriz con resolución de 20 m/pxl, a una con 10 m/pxl.
    Para ello divide cada píxel en 4 de igual valor.
    
    Retorna el vector con las nuevas dimensiones dadas por shape.
    '''
    
    out = np.zeros((1, shape[1], shape[2]), dtype="uint8")
    len_row = arr20.shape[2]*2
    
    if len_row >= shape[2]:
        for i in range(arr20.shape[1]):
            row = np.repeat(arr20[0, i, :], 2)
            if 2*i <=  shape[1]-1: 
                out[0, 2 * i, :] = row[0 : shape[2]]
            if 2*i + 1 <=  shape[1]-1: 
                out[0, 2 * i + 1, :] = row[0 : shape[2]]
                
        return out
    
    else:
        for i in range(arr20.shape[1]):
            row = np.repeat(arr20[0, i, :], 2)
            if 2*i <=  shape[1]-1: 
                out[0, 2 * i, :] = row
            if 2*i + 1 <=  shape[1]-1: 
                out[0, 2 * i + 1, :] = row
                
        return out

In [ ]:
def pxl30to10(arr30):
    '''
    Fragmenta una matriz con resolución de 30 m por píxel, a una con 10 m/pxl.
    Para ello divide cada píxel en nueve de igual valor
    '''
    
    out = np.zeros((3,arr30.shape[1]*3, arr30.shape[2]*3), dtype="uint8")
    
    for i in range(arr30.shape[1]):  
        row = np.repeat(arr30[0, i, :], 3)
        
        out[0, 3 * i, :] = row
        out[0, 3 * i + 1, :] = row
        out[0, 3 * i + 2, :] = row
        
        row1 = np.repeat(arr30[1, i, :], 3)
        
        out[1, 3 * i, :] = row1
        out[1, 3 * i + 1, :] = row1
        out[1, 3 * i + 2, :] = row1
        
        row2 = np.repeat(arr30[2, i, :], 3)
        
        out[2, 3 * i, :] = row2
        out[2, 3 * i + 1, :] = row2
        out[2, 3 * i + 2, :] = row2

    return out

In [ ]:
def cla_a_estimation(img_SCL, img_B03, img_B04, img_B05, log=0, sat='sentinel'):
    '''
    Estimación de los niveles de clorofila-a a partir de las bandas 3, 4 y 5 del Sentinel-2 MSI.
    La expresión utilizada es la calculada por Zhan et al.:
        124.94 * (b03_img + b05_img) / (b03_img + b04_img) - 115.35
    
    Para los datos de Landsat se estima el NDCI a partir de la expresión (Gabila Buma y Lee): 
        𝑁𝐷𝐶𝐼 = 0.6396 ∗ (𝑁𝐼𝑅 − 𝑅𝑒𝑑) /(𝑁𝐼𝑅 + 𝑅𝑒𝑑) + 0.4096
        
    Devuelve una lista con el valor medio estimado, su desviación típica y el log.
    '''
    if sat == 'sentinel':
        # Comprobación de que todas las imágenes tienen las mismas dimensiones
        shape_SCL = img_SCL.shape # (1,N,M). Resolucion de 20 m
        shape_B03 = img_B03.shape # (1,N,M)
        shape_B04 = img_B04.shape # (1,N,M)
        shape_B05 = img_B05.shape # La resolución de la banda 5 es de 20 m. Es necesario redimensionar la matriz

        if  (shape_B03[1:-1] != shape_B04[1:-1]):
            log = "Dimensiones no coincidentes entre resoluciones iguales (10m)"
            raise Exception("Error de tamaño en 10m")

        # Redimensionado. Convierte la matriz (1,n,m) en (1, shape[1], shape[2]) truncando si es necesario la matriz objetivo de (1,2n,2m)
        new_img_B05 = pxl20to10(img_B05, shape_B04)
        new_img_SCL = pxl20to10(img_SCL, shape_B04)
        
        if  (new_img_B05.shape[1:-1] != shape_B03[1:-1]) or (new_img_SCL.shape[1:-1] != shape_B03[1:-1]):
            log = "Dimensiones no coincidentes tras redimensionado"
            raise Exception("Error de tamaño en 20m tras redimensionado")

        # Calculo de la estimación de clorofila-a (en bruto).
        CHLa = np.zeros(shape_B04)
        for i in range(shape_B04[1]):
            for j in range(shape_B04[2]):
                
                # Se guardan en variables de numero flotante para evitar el overflowing del dato al ser en origen un entero.
                R705 = float(new_img_B05[0,i,j])
                R665 = float(img_B04[0,i,j])
                
                if R705 + R665 != 0:
                    CHLa[0,i,j] = 0.8489 * (R705 - R665) / (R705 + R665) - 0.1894
                else:
                    CHLa[0,i,j] = 0
                    
        # Filtrado según los píxeles de agua
        CHLa_MSK = CHLa[new_img_SCL == 4]
        if len(CHLa_MSK) == 0:
            mean_cla_a_value = 0
            std_cla_a_value = 0
        else:
            mean_cla_a_value = np.mean(CHLa_MSK)
            std_cla_a_value = np.std(CHLa_MSK)

        return [mean_cla_a_value, std_cla_a_value, log]
    
    else: # Landsat
        # imgB03 -> Rojo. 660nm
        # imgB04 -> NIR. 830nm
        # imgB05 -> None
        # Comprobación de que todas las imágenes tienen las mismas dimensiones. Todas tienen 30 m de resolución
        shape_SCL = img_SCL.shape # (N,M)
        shape_Red = img_B03.shape # (N,M)
        shape_NIR = img_B04.shape # (N,M)

        if (shape_SCL!=shape_Red) and (shape_SCL!=shape_NIR) and (shape_Red!=shape_NIR):
            log = "Dimensiones no coincidentes entre resoluciones iguales (30m)"
            raise Exception("Error de tamaño en 30m")
        
        # Calculo de la estimación de clorofila-a (en bruto). Indice NDCI.
        CHLa = np.zeros(shape_Red)
        for i in range(shape_Red[0]):
            for j in range(shape_Red[1]):
                
                # Se guardan en variables de numero flotante para evitar el overflowing del dato al ser en origen un entero.
                R865 = float(img_B04[i,j])
                R654 = float(img_B03[i,j])
                if R865 + R654 != 0:
                    CHLa[i,j] = 0.6396 * (R865 - R654) / (R865 + R654) - 0.4096
                else:
                    CHLa[i,j] = 0
        
        # Filtrado según los píxeles de agua
        CHLa_MSK = CHLa[img_SCL == 1]
        if len(CHLa_MSK) == 0:
            mean_cla_a_value = 0
            std_cla_a_value = 0
        else:
            mean_cla_a_value = np.mean(CHLa_MSK)
            std_cla_a_value = np.std(CHLa_MSK)
                
        return [mean_cla_a_value, std_cla_a_value, log]
        

In [ ]:
def visible_water_surface_estimation(img_SCL, resolution=20, sat='sentinel'):
    '''
    Calcula la superficie visible de agua en m^2. Puede ser útil para detectar pequeñas masas de agua cuyo dinámica de 
    propagación de HAB difiere de las masas más grandes.
    
    Devuelve un entero con la superficie de agua visible en el recorte.
    '''
    if sat=='sentinel':
        water_pixels = len(img_SCL[img_SCL == 4])
    else:
        water_pixels = len(img_SCL[img_SCL == 1])
    
    VWS = water_pixels*resolution**2
    
    return VWS

In [ ]:
def uniformar_imagen(img_TCI, sat='sentinel'):
    '''
    Redimensiona la imagen TCI.
    
    Devuelve la misma imágen truncando o añadiendo filas/columnas nulas según las dimensiones especificadas
    con el parámetro SHAPE.
    '''
    if sat == 'landsat':
        # Primero se dividen los píxeles de 30m en 9 píxeles de 10m.
        img_TCI = pxl30to10(img_TCI)
        shape_i = img_TCI.shape
    else:
        shape_i = img_TCI.shape
        
    new_img_TCI = np.zeros(SHAPE, dtype="uint8")
    
    num_filas = shape_i[1] - SHAPE[1]
    num_columnas = shape_i[2] - SHAPE[2]
    
    if num_columnas <= 0 and num_filas <= 0: # se añaden columnas y filas (ceros)
        new_img_TCI[:, 0:shape_i[1], 0:shape_i[2]] = img_TCI[:,:,:]

    elif num_columnas >= 0 and num_filas >= 0: # se truncan las filas y columnas sobrantes
        new_img_TCI = img_TCI[:, 0:SHAPE[1], 0:SHAPE[2]]

    elif num_columnas <= 0 and num_filas >= 0: # se truncan las filas y se añaden columnas
        new_img_TCI[:, :, 0:shape_i[2]]  = img_TCI[:, 0:SHAPE[1], :]
    
    elif num_columnas >= 0 and num_filas <= 0: # se añaden las filas y se truncan columnas
        new_img_TCI[:, 0:shape_i[1], :]  = img_TCI[:, :, 0:SHAPE[2]]

    return new_img_TCI

In [ ]:
def reconversion_target(predictions, labels, subset):
    '''
    Deshace la transformación lineal sobre la clase y calcula la precisión y las errores medios absolutos y cuadráticos
    promediados por la región donde se realizó la medición.
    
    Devuelve la precisión, el mse promediado por region, el mae promediado por region y una tupla con los rmse para cada region
    '''
    rounded_result = []
    acc_result = []
    err = []
    rmse = {'w':[], 'mw': [], 'ne':[], 's': []}
    rmae = {'w':[], 'mw': [], 'ne':[], 's': []}

    for i in range(len(predictions)):
        rounded_result.append((predictions[i] * 4 + 1).round())
        err.append(abs(rounded_result[-1]-labels.iloc[i]*4-1))

        if subset.region.iloc[i] == 'west':
            rmse['w'].append(err[-1]**2)
            rmae['w'].append(err[-1])
        elif subset.region.iloc[i] == 'south':
            rmse['s'].append(err[-1]**2)
            rmae['s'].append(err[-1])
        elif subset.region.iloc[i] == 'northeast':
            rmse['ne'].append(err[-1]**2)
            rmae['ne'].append(err[-1])
        elif subset.region.iloc[i] == 'midwest':
            rmse['mw'].append(err[-1]**2)
            rmae['mw'].append(err[-1])

        if rounded_result[-1] == labels.iloc[i]*4+1:
            acc_result.append(1)
        else:
            acc_result.append(0)
            
    acc = acc_result.count(1)/len(acc_result)
    rmae = (np.mean(rmae['w']) + np.mean(rmae['mw'])  + np.mean(rmae['ne'])  + np.mean(rmae['s'])) / 4
    rmse_val = (np.mean(rmse['w'])**0.5 + np.mean(rmse['mw'])**0.5  + np.mean(rmse['ne'])**0.5  + np.mean(rmse['s'])**0.5) / 4
    
    return acc, rmse_val, rmae, (np.mean(rmse['w'])**0.5,np.mean(rmse['mw'])**0.5,np.mean(rmse['ne'])**0.5,np.mean(rmse['s'])**0.5)

In [ ]:
def check_clouds(scl, sat='sentinel'):
    '''
    Calcula el porcentaje de nubosidad presente en la imagen diferenciando el origen de los datos.
    
    Devuelve un flotante con el porcentaje de nubosidad
    '''
    
    if sat=='sentinel':
        pxl_nubes = np.size( scl[ (scl == 3) | (scl == 8) | (scl == 9) | (scl == 10)] )
        total = np.size(scl)

        porc_nubosidad = pxl_nubes / total * 100
    else:
        cloud_image_array = CLEAR_BIT_FILTER(scl)
        pxl_nubes = np.size(cloud_image_array[cloud_image_array==1])
        total = np.size(scl)
        
        porc_nubosidad = pxl_nubes / total * 100
                
    return porc_nubosidad

In [ ]:
def mae_percent_error(pred, test):
    '''
    Calcula el porcentaje de mae tras deshacer la transformación lineal sobre la clase.
    
    Devuelve un flotante con el valor medio del error absoluto en términos porcentuales.
    '''
    
    maepe = []
    for i in range(len(predictions)):
        predi = pred[i] * 4 + 1
        testi = test.iloc[i] * 4 + 1

        maepe.append(100*abs(predi-testi)/testi)
        
    return np.mean(maepe)

## Paso 1: Preparar los datos de entrenamiento y prueba


In [ ]:
# carga de los metadatos
metadata = pd.read_csv(DATA_DIR / "metadata.csv")

# campo de tipo de dato del campo date a datetime
metadata.date = pd.to_datetime(metadata.date)

# estación en el momento de la medida
metadata["season"] = (
    metadata.date.dt.month.replace([12, 1, 2], "invierno")
    .replace([3, 4, 5], "primavera")
    .replace([6, 7, 8], "verano")
    .replace([9, 10, 11], "otoño")
)

# carga de las etiquetas
labels = pd.read_csv(DATA_DIR / "train_labels.csv")

metadata = metadata[metadata.usable=='ok']

# join entre los metadatos y las etiquetas
labels_and_metadata = labels.merge(
    metadata, how="left", left_on="uid", right_on="uid", validate="1:1"
)

### Transformación y carga de los datos extraídos de Sentinel-2 

In [ ]:
with open(SENTINEL_DATA_DIR / f"selected_items.txt", "rb") as f:
    selected_items = json.load(f)

In [ ]:
data_sentinel = pd.DataFrame(columns=["uid", "TCI", "cla_mean", "cla_std", "water_size", "T", "QI"])
log = {}
for row in tqdm(metadata.itertuples(), total=len(metadata)):
    try:
        log[row.uid] = 0
        # imagen a color verdadero (TCI). 10m de resolución
        with open(SENTINEL_DATA_DIR / f"visual/{row.uid}.npy", "rb") as f:
            img_TCI = np.load(f)
        # banda 3 (560 nm). 10m de resolución
        with open(SENTINEL_DATA_DIR / f"b03/{row.uid}.npy", "rb") as f:
            img_B03 = np.load(f)
        # banda 4 (665 nm). 10m de resolución
        with open(SENTINEL_DATA_DIR / f"b04/{row.uid}.npy", "rb") as f:
            img_B04 = np.load(f)
        # banda 5 (705 nm). 20m de resolución
        with open(SENTINEL_DATA_DIR / f"b05/{row.uid}.npy", "rb") as f:
            img_B05 = np.load(f)
        # Scene Classification Map. 20 m de resolución
        with open(SENTINEL_DATA_DIR / f"scl/{row.uid}.npy", "rb") as f:
            img_SCL = np.load(f)

        # Superficie de agua visible en la imagen
        visible_water_surface = visible_water_surface_estimation(img_SCL)

        # Calculo estimado de la concentración de clorofila a (a partir del indice NDCI)
        if visible_water_surface > 0:
            [cla_mean, cla_std, log[row.uid]] = cla_a_estimation(img_SCL, img_B03, img_B04, img_B05, log[row.uid])
        else:
            cla_mean = 0
            cla_std = 0
            log[row.uid] = 'No se detectaron píxeles de agua'

        # Redimensionamiento de la imagen a color verdadero para uniformar las dimensiones de todos los uid's
        img_TCI = uniformar_imagen(img_TCI)
        if img_TCI.shape != SHAPE:
            log[row.uid] = 'No uniformado.'
            print('err')
            raise Exception('No uniformado.')
            
        # Construcción del índice de calidad de la imagen
        score_time_delta = 100 * int(selected_items[row.uid]["time_diff"].split(" days 00:00:00")[0]) / 15
        score_nubosidad = 100 - check_clouds(img_SCL)
        score_resolucion = 100
        if visible_water_surface > 0:
            score_agua_detectada = 100
        else:
            score_agua_detectada = 0

        final_score = score_time_delta * 0.4 + score_nubosidad * 0.3 + score_resolucion * 0.1 + score_agua_detectada * 0.2
        
        new_row = pd.DataFrame({
            "uid": row.uid,
            "TCI": [img_TCI],
            "cla_mean": cla_mean,
            "cla_std": cla_std,
            "water_size":visible_water_surface,
            "T":-1, # Se completará posteriormente con los datos de HHRR NOAH
            "QI": final_score
        })
        
        data_sentinel = pd.concat([data_sentinel,new_row], ignore_index=True)
    except Exception as e:
        log[row.uid] = str(e)

In [ ]:
with open(SENTINEL_DATA_DIR / "TL_SENTINEL_log.txt", "w") as f:
    json.dump(log,f)

In [ ]:
data_sentinel.head()

Todas las imágenes deben tener las mismas dimensiones. Pese a emplear la misma resolución, al extraer las imágenes mediante la API de Microsoft Planetary Computer, se ajustan las dimensiones físicas a un número entero de píxeles produciéndose cierta variabilidad en la representación entre distintas imágenes según la aproximación que se realiza.

In [ ]:
i = 0
for ind,row in data_sentinel.iterrows():
    if row.TCI.shape != SHAPE:
        i+=1
i

Todo ok

A continuación se añade la información de temperatura obtenida de HRRR NOAA

In [ ]:
with open(SENTINEL_DATA_DIR / f"temperatures.txt", "rb") as f:
    temperatures = json.load(f)

In [ ]:
df_T = pd.DataFrame(pd.Series(temperatures), columns=['T'])

In [ ]:
df_data_sentinel = data_sentinel.merge(df_T, how='left', left_on='uid', right_index=True).drop('T_x', axis=1).rename(columns={"T_y":"T"})

In [ ]:
df_data_sentinel.head()

### Transformación y carga de los datos extraídos de Landsat 

In [ ]:
with open(LANDSAT_DATA_DIR / f"selected_items.txt", "rb") as f:
    selected_items = json.load(f)

In [ ]:
with open(LANDSAT_DATA_DIR / f"selected_items7.txt", "rb") as f:
    selected_items7 = json.load(f)

In [ ]:
data_landsat = pd.DataFrame(columns=["uid", "TCI", "cla_mean", "cla_std", "water_size", "T", "QI"])
log = {}
for row in tqdm(metadata.itertuples(), total=len(metadata)):
    try:
        log[row.uid] = 0
        # imagen a color verdadero (TCI). 30m de resolución [Red, Green, Blue]
        with open(LANDSAT_DATA_DIR / f"visual/{row.uid}.npy", "rb") as f:
            img_TCI = np.load(f)
        # Banda lwir. Permite estimar la temperatura mediante la siguiente transformación lineal: temp * 0.00341802 + 149.
        # 30m de resolución
        with open(LANDSAT_DATA_DIR / f"temperature/{row.uid}.npy", "rb") as f:
            img_T = np.load(f)
        # Banda de Quality Assesment de los píxeles. Incluye información de la nubosidad así de como píxeles de agua. 
        # 30m de resolución
        with open(LANDSAT_DATA_DIR / f"cloud/{row.uid}.npy", "rb") as f:
            img_QA = np.load(f)
        # banda NIR (Near Infrarred). Se usa en la estimación de cla-a. 30m de resolución
        with open(LANDSAT_DATA_DIR / f"nir/{row.uid}.npy", "rb") as f:
            img_NIR = np.load(f)

        # Superficie de agua visible en la imagen
        water_img = WATER_BIT_FILTER(img_QA)
        visible_water_surface = visible_water_surface_estimation(water_img, 30, 'landsat')

        # Calculo estimado de la concentración de clorofila a
        if visible_water_surface > 0:
            [cla_mean, cla_std, log[row.uid]] = cla_a_estimation(water_img[0], img_TCI[0], img_NIR[0], None, log[row.uid], 'landsat')
        else:
            cla_mean = 0
            cla_std = 0
            log[row.uid] = 'No se detectaron píxeles de agua'

        # Estimación la temperatura media a partir de la señal lwir de LANDSAT. Aplicando la transformación lineal dada por las especificaciones
        # del producto: temp * 0.00341802 + 149
        temperature = np.mean(img_T) * 0.00341802 + 149
        
         # Construcción del índice de calidad de la imagen
        if row.uid in selected_items:
            score_time_delta = 100 * int(selected_items[row.uid]["time_diff"].split(" days 00:00:00")[0]) / 15
        else:
            score_time_delta = 100 * int(selected_items7[row.uid]["time_diff"].split(" days 00:00:00")[0]) / 15
        score_nubosidad = 100 - check_clouds(img_QA, sat='landsat')
        score_resolucion = 0
        if visible_water_surface > 0:
            score_agua_detectada = 100
        else:
            score_agua_detectada = 0

        final_score = score_time_delta * 0.4 + score_nubosidad * 0.3 + score_resolucion * 0.1 + score_agua_detectada * 0.2

        # Redimensionamiento de la imagen a color verdadero para uniformar las dimensiones de todos los uid's
        img_TCI = uniformar_imagen(img_TCI, 'landsat')

        new_row = pd.DataFrame({
            "uid": row.uid,
            "TCI": [img_TCI],
            "cla_mean": cla_mean,
            "cla_std": cla_std,
            "water_size": visible_water_surface,
            "T": temperature,
            "QI": final_score
        })

        data_landsat = pd.concat([data_landsat,new_row], ignore_index=True)
    except Exception as e:
        log[row.uid] = str(e)

In [ ]:
with open(LANDSAT_DATA_DIR / "TL_LANDSAT_log.txt", "w") as f:
    json.dump(log,f)

In [ ]:
data_landsat.head()

Todas las imágenes deben tener las mismas dimensiones

In [ ]:
i=0
for ind,row in data_landsat.iterrows():
    if row.TCI.shape != SHAPE:
        i+=1
i

Ok

Concatenación de los datos extraídos de sentinel y landsat

In [ ]:
df_data_sentinel

In [ ]:
df_data = pd.concat([data_landsat,df_data_sentinel], ignore_index=True)

In [ ]:
df_data_qi25 = df_data[df_data.QI>=25]

In [ ]:
df_data_qi50 = df_data[df_data.QI>=50]

## Paso 2: Preparación de los datos

In [ ]:
df_final = labels_and_metadata.merge(
    df_data, how="inner", left_on="uid", right_on="uid", validate="1:1"
)

Split del dataset (estratificado por la clase objetivo)

In [ ]:
train_subset, test_subset = train_test_split(df_final, test_size=0.2, random_state=10, stratify=df_final.severity)

Onehot encoding del campo season

In [ ]:
season_map = {'invierno': 0, 'primavera':1, 'verano': 2, 'otoño': 3} 
train_subset[['invierno', 'primavera', "verano", "otoño"]] = to_categorical(train_subset.season.map(season_map))

In [ ]:
test_subset[['invierno', 'primavera', "verano", "otoño"]] = to_categorical(test_subset.season.map(season_map))

Onehot encoding del campo region

In [ ]:
region_map = {'south': 0, 'west':1, 'midwest': 2, 'northeast': 3} 
train_subset[['south', 'west', "midwest", "northeast"]] = to_categorical(train_subset.region.map(region_map))

In [ ]:
test_subset[['south', 'west', "midwest", "northeast"]] = to_categorical(test_subset.region.map(region_map))

Separación de los inputs en imagen y características

In [ ]:
train_features = train_subset[["cla_mean", "cla_std", "water_size", "T", "invierno", 'primavera', "verano", "otoño", 'south', 'west', "midwest", "northeast"]]
test_features = test_subset[["cla_mean", "cla_std", "water_size", "T", "invierno", 'primavera', "verano", "otoño", 'south', 'west', "midwest", "northeast"]]

train_img = train_subset[["TCI"]]
test_img = test_subset[["TCI"]]

# El mínimo nivel de severidad es 1, se reducen todos uno para empezar desde 0 y aplicar la activación sigmoide en la capa final
train_labels = train_subset.severity-1
test_labels = test_subset.severity-1                    

In [ ]:
len_train = len(train_subset)
len_test = len(test_subset)

Reescalado de las imágenes

In [ ]:
# Se elimina la primera dimensión de las imágenes empleada al empaquetar la imagen durante la carga dentro del df
train_img2 = train_img.TCI.apply(lambda item:  np.array([item[0], item[1], item[2]]))
train_img3 = []
for i in range(len_train):
    train_img3.append(train_img2.iloc[i])
    
train_img4 = np.reshape(train_img3,(len_train,42,42,3))
train_img4 = train_img4.astype('float32') / 255 # Se reescalan de 0 a 1 los valores de la imagen

# Se elimina la primera dimensión de las imágenes empleada al empaquetar la imagen durante la carga dentro del df
test_img2 = test_img.TCI.apply(lambda item:  np.array([item[0], item[1], item[2]]))
test_img3 = []
for i in range(len_test):
    test_img3.append(test_img2.iloc[i])

test_img4 = np.reshape(test_img3,(len_test,42,42,3))
test_img4 = test_img4.astype('float32') / 255 # Se reescalan de 0 a 1 los valores de la imagen

Escalado features

In [ ]:
## Temperatura
# Máximo y mínimo de train
maxs = np.max(train_features['T'], axis=0)
mins = np.min(train_features['T'], axis=0)
ranges = maxs - mins

# Escalado entre 0 y 1 de la media de concentracion de cla-a. SE NORMALIZA CON LOS VALORES EXTREMOS DE TRAIN PARA NO FILTRAR INFORMACIÓN AL MODELO.
train_features['T'] = (train_features['T'] - mins) / ranges
test_features['T'] = (test_features['T'] - mins) / ranges

# Cla-a valor medio
# Máximo y mínimo de train
maxs = np.max(train_features.cla_mean, axis=0)
mins = np.min(train_features.cla_mean, axis=0)
ranges = maxs - mins

# Escalado entre 0 y 1 de la media de concentracion de cla-a. SE NORMALIZA CON LOS VALORES EXTREMOS DE TRAIN PARA NO FILTRAR INFORMACIÓN AL MODELO.
train_features.cla_mean = (train_features.cla_mean - mins) / ranges
test_features.cla_mean = (test_features.cla_mean - mins) / ranges

# Cla-a std
# Máximo y mínimo de train
maxs = np.max(train_features.cla_std, axis=0)
mins = np.min(train_features.cla_std, axis=0)
ranges = maxs - mins

# Escalado entre 0 y 1 de la media de concentracion de cla-a. SE NORMALIZA CON LOS VALORES EXTREMOS DE TRAIN PARA NO FILTRAR INFORMACIÓN AL MODELO.
train_features.cla_std = (train_features.cla_std - mins) / ranges
test_features.cla_std = (test_features.cla_std - mins) / ranges

# Superficie visible de agua
# Máximo y mínimo de train
maxs = np.max(train_features.water_size, axis=0)
mins = np.min(train_features.water_size, axis=0)
ranges = maxs - mins

# Escalado entre 0 y 1 de la media de concentracion de cla-a. SE NORMALIZA CON LOS VALORES EXTREMOS DE TRAIN PARA NO FILTRAR INFORMACIÓN AL MODELO.
train_features.water_size = (train_features.water_size - mins) / ranges
test_features.water_size = (test_features.water_size - mins) / ranges

Construcción del vector de features

In [ ]:
train_features2 = np.array([train_features['cla_mean'], train_features['cla_std'], train_features['water_size'], train_features['T'], train_features["invierno"], train_features['primavera'], train_features["verano"], train_features["otoño"], train_features['south'], train_features['west'], train_features["midwest"], train_features["northeast"]]).T
train_features3 = train_features2.astype('float32')

test_features2 = np.array([test_features['cla_mean'], test_features['cla_std'], test_features['water_size'], test_features['T'], test_features["invierno"], test_features['primavera'], test_features["verano"], test_features["otoño"], test_features['south'], test_features['west'], test_features["midwest"], test_features["northeast"]]).T
test_features3 = test_features2.astype('float32')

Escalado del target

In [ ]:
test_labels = test_labels.astype('float32') / np.max(train_labels, axis=0)
train_labels = train_labels.astype('float32') / np.max(train_labels, axis=0)

In [ ]:
number_of_features = len(train_features3[0])

## Paso 3: Creacion y evaluación de los modelos

### Modelo 1: Simple

In [ ]:
# Agregar capas de entrada
img_input = layers.Input(shape=(SHAPE[1], SHAPE[2], SHAPE[0]))

# Agregar capas convolucionales. Procesado imagen
x = layers.Conv2D(32, (3, 3), activation='relu')(img_input)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)

# Aplanar los datos y agregar capas densas (fully connected)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='tanh')(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.3)(x)

output = layers.Dense(1, activation="sigmoid")(x)

model_simple = models.Model(img_input, output)

model_simple.summary()

In [ ]:
checkpoint_filepath = DATA_DIR / 'base_weights.{epoch:02d}-{val_loss:.4f}.hdf5'

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [ ]:
model_simple.compile(optimizer='rmsprop',
              loss='mae',
              metrics=['mse'])

In [ ]:
history_base_model = model_simple.fit(x=train_img4, y=train_labels, epochs=50, validation_data=(test_img4, test_labels), callbacks=model_checkpoint_callback)

In [ ]:
model_simple.load_weights(DATA_DIR / 'base_weights.43-0.1512.hdf5')

In [ ]:
model_simple.save("model_simple.h5")

In [ ]:
# model_simple = keras.models.load_model("model_simple.h5")

In [ ]:
flg,ax = plt.subplots()
ax.plot(range(50), history_base_model.history['loss'], label='train')
ax.plot(range(50),history_base_model.history['val_loss'], label='test')
ax.set_xlabel('Epochs')
ax.set_ylabel('MAE')
ax.legend()

In [ ]:
flg,ax = plt.subplots()
ax.plot(range(50), history_base_model.history['mse'], label='train')
ax.plot(range(50),history_base_model.history['val_mse'], label='test')
ax.set_xlabel('Epochs')
ax.set_ylabel('MSE')
ax.legend()

In [ ]:
predictions = model_simple.predict(train_img4)

In [ ]:
rounded_result, acc, rmse, rmae = reconversion_target(predictions, train_labels, train_subset)

In [ ]:
acc, rmse, rmae

In [ ]:
mae_percent_error(predictions, train_labels)

In [ ]:
predictions = model_simple.predict(test_img4)

In [ ]:
rounded_result, acc, rmse, rmae = reconversion_target(predictions, test_labels, test_subset)

In [ ]:
acc, rmse, rmae

In [ ]:
mae_percent_error(predictions, test_labels)

In [ ]:
evaluations = model_simple.evaluate(train_img4, train_labels)

In [ ]:
evaluations = model_simple.evaluate(test_img4, test_labels)

### Modelo 2: Convolucional + features en paralelo 

In [ ]:
# Agregar capas de entrada
img_input = layers.Input(shape=(SHAPE[1], SHAPE[2], SHAPE[0]))
features_input = layers.Input(shape=(number_of_features,))

# Agregar capas convolucionales. Procesado imagen
x = layers.Conv2D(32, (3, 3), activation='relu')(img_input)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)

# Aplanar los datos y agregar capas densas (fully connected)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)
y = layers.Dropout(0.3)(x)

# Se introduce el output de la última capa convolucional como input junto con las features calculadas
x = layers.Dense(128, activation='relu')(features_input)
x = layers.Dense(64, activation='relu')(x)
x = layers.concatenate([y, x])
x = layers.Dense(64, activation='relu')(x)

output = layers.Dense(1, activation="sigmoid")(x)

model_features = models.Model([img_input, features_input], output)

model_features.summary()

In [ ]:
checkpoint_filepath = DATA_DIR / 'features_weights.{epoch:02d}-{val_loss:.4f}.hdf5'

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [ ]:
model_features.compile(optimizer='rmsprop',
              loss='mae',
              metrics=['mse'])

In [ ]:
history_features_model = model_features.fit(x=[train_img4, train_features3], y=train_labels, epochs=100, validation_data=([test_img4, test_features3], test_labels), callbacks=model_checkpoint_callback)

In [ ]:
model_features.load_weights(DATA_DIR / 'features_weights.65-0.1253.hdf5')

In [ ]:
model_features.save("model_features.h5")

In [ ]:
# model_features = keras.models.load_model("model_features.h5")

In [ ]:
flg,ax = plt.subplots()
ax.plot(range(100), history_features_model.history['loss'], label='train')
ax.plot(range(100),history_features_model.history['val_loss'], label='test')
ax.set_xlabel('Epochs')
ax.set_ylabel('MSE')
ax.legend()

In [ ]:
flg,ax = plt.subplots()
ax.plot(range(100), history_features_model.history['mse'], label='train')
ax.plot(range(100),history_features_model.history['val_mse'], label='test')
ax.set_xlabel('Epochs')
ax.set_ylabel('MSE')
ax.legend()

In [ ]:
predictions = model_features.predict([train_img4, train_features3])

In [ ]:
rounded_result, acc, rmse, rmae = reconversion_target(predictions, train_labels, train_subset)

In [ ]:
acc, rmse, rmae

In [ ]:
mae_percent_error(predictions, train_labels)

In [ ]:
predictions = model_features.predict([test_img4, test_features3])

In [ ]:
reconversion_target(predictions, test_labels, test_subset)

In [ ]:
acc, rmse, rmae

In [ ]:
binary_acc.count(1) / (binary_acc.count(1) + binary_acc.count(0))

In [ ]:
mae_percent_error(predictions, test_labels)

In [ ]:
evaluations = model_features.evaluate([train_img4, train_features3], train_labels)

In [ ]:
evaluations = model_features.evaluate([test_img4, test_features3], test_labels)

### Modelo 3: Inception

In [ ]:
# Agregar capas de entrada
img_input = layers.Input(shape=(SHAPE[1], SHAPE[2], SHAPE[0]))

# Agregar capas convolucionales. Procesado imagen
x1 = layers.Conv2D(16, (1, 1), strides = 2, activation='relu')(img_input)
x1 = layers.Cropping2D(cropping=((2, 1), (2, 1)))(x1)

x2 = layers.Conv2D(32, (1, 1), strides = 1, activation='relu')(img_input)
x2 = layers.Conv2D(16, (3, 3), strides = 2, activation='relu')(x2)
x2 = layers.Cropping2D(cropping=((1, 1), (1, 1)))(x2)

x3 = layers.AveragePooling2D((3, 3), strides = 2)(img_input)
x3 = layers.Conv2D(16, (3, 3), strides = 1, activation='relu')(x3)

x4 = layers.Conv2D(32, (1, 1), strides = 1, activation='relu')(img_input)
x4 = layers.Conv2D(16, (3, 3), strides = 1, activation='relu')(x4)
x4 = layers.Conv2D(16, (3, 3), strides = 2, activation='relu')(x4)
x4 = layers.Cropping2D(cropping=((0, 1), (0, 1)))(x4)

x = layers.concatenate([x1, x2, x3, x4])
x = layers.Flatten()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(32, activation='relu')(x)

output = layers.Dense(1, activation="sigmoid")(x)

model_inception = models.Model(img_input, output)

model_inception.summary()

In [ ]:
checkpoint_filepath = DATA_DIR / 'inception_weights.{epoch:02d}-{val_loss:.4f}.hdf5'

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [ ]:
model_inception.compile(optimizer='rmsprop',
              loss='mae',
              metrics=['mse'])

In [ ]:
history_inception_model = model_inception.fit(x=train_img4, y=train_labels, epochs=50, validation_data=(test_img4, test_labels), callbacks=model_checkpoint_callback)

In [ ]:
model_inception.load_weights(DATA_DIR / 'inception_weights.09-0.1353.hdf5')

In [ ]:
model_inception.save("model_inception.h5")

In [ ]:
# model_inception = keras.models.load_model("model_inception.h5")

In [ ]:
flg,ax = plt.subplots()
ax.plot(range(50), history_inception_model.history['loss'], label='train')
ax.plot(range(50),history_inception_model.history['val_loss'], label='test')
ax.set_xlabel('Epochs')
ax.set_ylabel('MAE')
ax.legend()

In [ ]:
flg,ax = plt.subplots()
ax.plot(range(50), history_inception_model.history['mse'], label='train')
ax.plot(range(50),history_inception_model.history['val_mse'], label='test')
ax.set_xlabel('Epochs')
ax.set_ylabel('MSE')
ax.legend()

In [ ]:
predictions = model_inception.predict(train_img4)

In [ ]:
rounded_result, acc, rmse, rmae = reconversion_target(predictions, train_labels, train_subset)

In [ ]:
acc, rmse, rmae

In [ ]:
mae_percent_error(predictions, train_labels)

In [ ]:
predictions = model_inception.predict(test_img4)

In [ ]:
rounded_result, acc, rmse, rmae = reconversion_target(predictions, test_labels, test_subset)

In [ ]:
acc, rmse, rmae

In [ ]:
mae_percent_error(predictions, test_labels)

In [ ]:
evaluations = model_inception.evaluate(test_img4, test_labels)

In [ ]:
evaluations = model_inception.evaluate(train_img4, train_labels)

### Modelo 4: Inception + features en paralelo

In [ ]:
# Agregar capas de entrada
img_input = layers.Input(shape=(SHAPE[1], SHAPE[2], SHAPE[0]))
features_input = layers.Input(shape=(number_of_features,))

# Agregar capas convolucionales. Procesado imagen
x1 = layers.Conv2D(16, (1, 1), strides = 2, activation='relu')(img_input)
x1 = layers.Cropping2D(cropping=((2, 1), (2, 1)))(x1)

x2 = layers.Conv2D(32, (1, 1), strides = 1, activation='relu')(img_input)
x2 = layers.Conv2D(16, (3, 3), strides = 2, activation='relu')(x2)
x2 = layers.Cropping2D(cropping=((1, 1), (1, 1)))(x2)

x3 = layers.AveragePooling2D((3, 3), strides = 2)(img_input)
x3 = layers.Conv2D(16, (3, 3), strides = 1, activation='relu')(x3)

x4 = layers.Conv2D(32, (1, 1), strides = 1, activation='relu')(img_input)
x4 = layers.Conv2D(16, (3, 3), strides = 1, activation='relu')(x4)
x4 = layers.Conv2D(16, (3, 3), strides = 2, activation='relu')(x4)
x4 = layers.Cropping2D(cropping=((0, 1), (0, 1)))(x4)

x = layers.concatenate([x1, x2, x3, x4])
x = layers.Flatten()(x)
x = layers.Dropout(0.3)(x)
y = layers.Dense(32, activation='relu')(x)

# Se introduce el output de la última capa convolucional como input junto con las features calculadas
x = layers.Dense(64, activation='relu')(features_input)
x = layers.Dense(32, activation='relu')(x)
x = layers.concatenate([y, x])
x = layers.Dense(64, activation='relu')(x)

output = layers.Dense(1, activation="sigmoid")(x)

model_inception_features = models.Model([img_input,features_input], output)

model_inception_features.summary()

In [ ]:
checkpoint_filepath = DATA_DIR / 'inception_features_weights.{epoch:02d}-{val_loss:.4f}.hdf5'

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [ ]:
model_inception_features.compile(optimizer='rmsprop',
              loss='mae',
              metrics=['mse'])

In [ ]:
history_inception_features_model = model_inception_features.fit(x=[train_img4, train_features3], y=train_labels, epochs=50, validation_data=([test_img4, test_features3], test_labels), callbacks=model_checkpoint_callback)

In [ ]:
model_inception_features.load_weights(DATA_DIR / 'inception_features_weights.10-0.0334.hdf5')

In [ ]:
model_inception_features.save("model_inception_features.h5")

In [ ]:
# model_inception_features = keras.models.load_model("model_inception_features.h5")

In [ ]:
flg,ax = plt.subplots()
ax.plot(range(50), history_inception_features_model.history['loss'], label='train')
ax.plot(range(50),history_inception_features_model.history['val_loss'], label='test')
ax.set_xlabel('Epochs')
ax.set_ylabel('MAE')
ax.legend()

In [ ]:
flg,ax = plt.subplots()
ax.plot(range(50), history_inception_features_model.history['mse'], label='train')
ax.plot(range(50),history_inception_features_model.history['val_mse'], label='test')
ax.set_xlabel('Epochs')
ax.set_ylabel('MSE')
ax.legend()

In [ ]:
predictions = model_inception_features.predict([train_img4, train_features3])

In [ ]:
rounded_result, acc, rmse, rmae = reconversion_target(predictions, train_labels, train_subset)

In [ ]:
acc, rmse, rmae

In [ ]:
mae_percent_error(predictions, train_labels)

In [ ]:
predictions = model_inception_features.predict([test_img4, test_features3])

In [ ]:
rounded_result, acc, rmse, rmae = reconversion_target(predictions, test_labels, test_subset)

In [ ]:
acc, rmse, rmae

In [ ]:
mae_percent_error(predictions, test_labels)

In [ ]:
evaluations = model_inception_features.evaluate([train_img4, train_features3], train_labels)

In [ ]:
evaluations = model_inception_features.evaluate([test_img4, test_features3], test_labels)

## Paso 4: Optimizacion del modelo 2

### Reducción. 2 capa conv

In [ ]:
# Agregar capas de entrada
img_input = layers.Input(shape=(SHAPE[1], SHAPE[2], SHAPE[0]))
features_input = layers.Input(shape=(number_of_features,))

# Agregar capas convolucionales. Procesado imagen
x = layers.Conv2D(32, (3, 3), activation='relu')(img_input)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)


# Aplanar los datos y agregar capas densas (fully connected)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)
y = layers.Dropout(0.3)(x)

# Se introduce el output de la última capa convolucional como input junto con las features calculadas
x = layers.Dense(128, activation='relu')(features_input)
x = layers.Dense(64, activation='relu')(x)
x = layers.concatenate([y, x])
x = layers.Dense(64, activation='relu')(x)

output = layers.Dense(1, activation="sigmoid")(x)

model_features_red_2lay = models.Model([img_input, features_input], output)

model_features_red_2lay.summary()

In [ ]:
checkpoint_filepath = DATA_DIR / 'model_features_red_2lay.{epoch:02d}-{val_loss:.4f}.hdf5'

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [ ]:
model_features_red_2lay.compile(optimizer='rmsprop',
              loss='mae',
              metrics=['mse'])

In [ ]:
history_model_features_red_2lay = model_features_red_2lay.fit(x=[train_img4, train_features3], y=train_labels, epochs=50, validation_data=([test_img4, test_features3], test_labels), callbacks=model_checkpoint_callback)

In [ ]:
model_features_red_2lay.load_weights(DATA_DIR / 'model_features_red_2lay.49-0.1249.hdf5')

In [ ]:
model_features_red_2lay.save("model_features_red_2lay.h5")

In [ ]:
# model_features = keras.models.load_model("model_features.h5")

In [ ]:
flg,ax = plt.subplots()
ax.plot(range(50), history_model_features_red_2lay.history['loss'], label='train')
ax.plot(range(50),history_model_features_red_2lay.history['val_loss'], label='test')
ax.set_xlabel('Epochs')
ax.set_ylabel('MAE')
ax.legend()

In [ ]:
flg,ax = plt.subplots()
ax.plot(range(50), history_model_features_red_2lay.history['mse'], label='train')
ax.plot(range(50),history_model_features_red_2lay.history['val_mse'], label='test')
ax.set_xlabel('Epochs')
ax.set_ylabel('MSE')
ax.legend()

In [ ]:
predictions = model_features_red_2lay.predict([train_img4, train_features3])

In [ ]:
rounded_result, acc, rmse, rmae = reconversion_target(predictions, train_labels, train_subset)

In [ ]:
acc, rmse, rmae

In [ ]:
mae_percent_error(predictions, train_labels)

In [ ]:
predictions = model_features_red_2lay.predict([test_img4, test_features3])

In [ ]:
rounded_result, acc, rmse, rmae = reconversion_target(predictions, test_labels, test_subset)

In [ ]:
acc, rmse, rmae

In [ ]:
mae_percent_error(predictions, test_labels)

In [ ]:
evaluations = model_features_red_2lay.evaluate([test_img4, test_features3], test_labels)

In [ ]:
evaluations = model_features_red_2lay.evaluate([train_img4, train_features3], train_labels)

### Reducción. 1 capa conv

In [ ]:
# Agregar capas de entrada
img_input = layers.Input(shape=(SHAPE[1], SHAPE[2], SHAPE[0]))
features_input = layers.Input(shape=(number_of_features,))

# Agregar capas convolucionales. Procesado imagen
x = layers.Conv2D(64, (3, 3), activation='relu')(img_input)

# Aplanar los datos y agregar capas densas (fully connected)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)
y = layers.Dropout(0.3)(x)

# Se introduce el output de la última capa convolucional como input junto con las features calculadas
x = layers.Dense(128, activation='relu')(features_input)
x = layers.Dense(64, activation='relu')(x)
x = layers.concatenate([y, x])
x = layers.Dense(64, activation='relu')(x)

output = layers.Dense(1, activation="sigmoid")(x)

model_features_red_1lay = models.Model([img_input, features_input], output)

model_features_red_1lay.summary()

In [ ]:
checkpoint_filepath = DATA_DIR / 'model_features_red_1lay.{epoch:02d}-{val_loss:.4f}.hdf5'

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [ ]:
model_features_red_1lay.compile(optimizer='rmsprop',
              loss='mae',
              metrics=['mse'])

In [ ]:
history_model_features_red_1lay = model_features_red_1lay.fit(x=[train_img4, train_features3], y=train_labels, epochs=50, validation_data=([test_img4, test_features3], test_labels), callbacks=model_checkpoint_callback)

In [ ]:
model_features_red_1lay.load_weights(DATA_DIR / 'model_features_red_1lay.47-0.1268.hdf5')

In [ ]:
model_features_red_1lay.save("model_features_red_1lay.h5")

In [ ]:
# model_features = keras.models.load_model("model_features.h5")

In [ ]:
plt.plot(range(50), history_model_features_red_1lay.history['loss'],range(50),history_model_features_red_1lay.history['val_loss'])
plt.xlabel('Epochs')
plt.ylabel('MSE')

In [ ]:
predictions = model_features_red_1lay.predict([train_img4, train_features3])

In [ ]:
rounded_result, acc, rmse, rmae = reconversion_target(predictions, train_labels, train_subset)

In [ ]:
acc, rmse, rmae

In [ ]:
mae_percent_error(predictions, train_labels)

In [ ]:
predictions = model_features_red_1lay.predict([test_img4, test_features3])

In [ ]:
rounded_result, acc, rmse, rmae = reconversion_target(predictions, test_labels, test_subset)

In [ ]:
acc, rmse, rmae

In [ ]:
mae_percent_error(predictions, test_labels)

In [ ]:
evaluations = model_features_red_1lay.evaluate([train_img4, train_features3], train_labels)

In [ ]:
evaluations = model_features_red_1lay.evaluate([test_img4, test_features3], test_labels)

### Reducción. 4 capa conv

In [ ]:
# Agregar capas de entrada
img_input = layers.Input(shape=(SHAPE[1], SHAPE[2], SHAPE[0]))
features_input = layers.Input(shape=(number_of_features,))

# Agregar capas convolucionales. Procesado imagen
x = layers.Conv2D(64, (3, 3), activation='relu')(img_input)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)

# Aplanar los datos y agregar capas densas (fully connected)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)
y = layers.Dropout(0.3)(x)

# Se introduce el output de la última capa convolucional como input junto con las features calculadas
x = layers.Dense(128, activation='relu')(features_input)
x = layers.Dense(64, activation='relu')(x)
x = layers.concatenate([y, x])
x = layers.Dense(64, activation='relu')(x)

output = layers.Dense(1, activation="sigmoid")(x)

model_features_red_4lay = models.Model([img_input, features_input], output)

model_features_red_4lay.summary()

In [ ]:
checkpoint_filepath = DATA_DIR / 'model_features_red_4lay.{epoch:02d}-{val_loss:.4f}.hdf5'

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [ ]:
model_features_red_4lay.compile(optimizer='rmsprop',
              loss='mse',
              metrics=['mae', tf.keras.metrics.mean_absolute_percentage_error])

In [ ]:
history_model_features_red_4lay = model_features_red_4lay.fit(x=[train_img4, train_features3], y=train_labels, epochs=50, validation_data=([test_img4, test_features3], test_labels), callbacks=model_checkpoint_callback)

In [ ]:
model_features_red_4lay.load_weights(DATA_DIR / 'model_features_red_4lay.15-0.0342.hdf5')

In [ ]:
model_features_red_4lay.save("model_features_red_4lay.h5")

In [ ]:
# model_features_red_4lay = keras.models.load_model("model_features_red_4lay.h5")

In [ ]:
plt.plot(range(50), history_model_features_red_4lay.history['loss'],range(50),history_model_features_red_4lay.history['val_loss'])
plt.xlabel('Epochs')
plt.ylabel('MSE')

In [ ]:
predictions = model_features_red_4lay.predict([train_img4, train_features3])

In [ ]:
rounded_result, acc, rmse, rmae = reconversion_target(predictions, train_labels, train_subset)

In [ ]:
acc, rmse, rmae

In [ ]:
mae_percent_error(predictions, train_labels)

In [ ]:
predictions = model_features_red_4lay.predict([test_img4, test_features3])

In [ ]:
rounded_result, acc, rmse, rmae = reconversion_target(predictions, test_labels, test_subset)

In [ ]:
acc, rmse, rmae

In [ ]:
mae_percent_error(predictions, test_labels)

In [ ]:
evaluations = model_features_red_4lay.evaluate([train_img4, train_features3], train_labels)

In [ ]:
evaluations = model_features_red_4lay.evaluate([test_img4, test_features3], test_labels)

### Neuronas. x2

In [ ]:
# Agregar capas de entrada
img_input = layers.Input(shape=(SHAPE[1], SHAPE[2], SHAPE[0]))
features_input = layers.Input(shape=(number_of_features,))

# Agregar capas convolucionales. Procesado imagen
x = layers.Conv2D(32*2, (3, 3), activation='relu')(img_input)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64*2, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64*2, (3, 3), activation='relu')(x)

# Aplanar los datos y agregar capas densas (fully connected)
x = layers.Flatten()(x)
x = layers.Dense(128*2, activation='relu')(x)
x = layers.Dense(64*2, activation='relu')(x)
y = layers.Dropout(0.3)(x)

# Se introduce el output de la última capa convolucional como input junto con las features calculadas
x = layers.Dense(128*2, activation='relu')(features_input)
x = layers.Dense(64*2, activation='relu')(x)
x = layers.concatenate([y, x])
x = layers.Dense(64*2, activation='relu')(x)

output = layers.Dense(1, activation="sigmoid")(x)

model_features_neur2 = models.Model([img_input, features_input], output)

model_features_neur2.summary()

In [ ]:
checkpoint_filepath = DATA_DIR / 'model_features_neur2.{epoch:02d}-{val_loss:.4f}.hdf5'

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [ ]:
model_features_neur2.compile(optimizer='rmsprop',
              loss='mse',
              metrics=['mae', tf.keras.metrics.mean_absolute_percentage_error])

In [ ]:
history_model_features_neur2 = model_features_neur2.fit(x=[train_img4, train_features3], y=train_labels, epochs=50, validation_data=([test_img4, test_features3], test_labels), callbacks=model_checkpoint_callback)

In [ ]:
model_features_neur2.load_weights(DATA_DIR / 'model_features_neur2.15-0.0327.hdf5')

In [ ]:
model_features_neur2.save("model_features_neur2.h5")

In [ ]:
# model_features_neur2 = keras.models.load_model("model_features_neur2.h5")

In [ ]:
plt.plot(range(50), history_model_features_neur2.history['loss'],range(50),history_model_features_neur2.history['val_loss'])
plt.xlabel('Epochs')
plt.ylabel('MSE')

In [ ]:
predictions = model_features_neur2.predict([train_img4, train_features3])

In [ ]:
rounded_result, acc, rmse, rmae = reconversion_target(predictions, train_labels, train_subset)

In [ ]:
acc, rmse, rmae

In [ ]:
mae_percent_error(predictions, train_labels)

In [ ]:
predictions = model_features_neur2.predict([test_img4, test_features3])

In [ ]:
rounded_result, acc, rmse, rmae = reconversion_target(predictions, test_labels, test_subset)

In [ ]:
acc, rmse, rmae

In [ ]:
mae_percent_error(predictions, test_labels)

In [ ]:
evaluations = model_features_neur2.evaluate([train_img4, train_features3], train_labels)

In [ ]:
evaluations = model_features_neur2.evaluate([test_img4, test_features3], test_labels)

### Neuronas. x 1/2

In [ ]:
# Agregar capas de entrada
img_input = layers.Input(shape=(SHAPE[1], SHAPE[2], SHAPE[0]))
features_input = layers.Input(shape=(number_of_features,))

# Agregar capas convolucionales. Procesado imagen
x = layers.Conv2D(32/2, (3, 3), activation='relu')(img_input)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64/2, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64/2, (3, 3), activation='relu')(x)

# Aplanar los datos y agregar capas densas (fully connected)
x = layers.Flatten()(x)
x = layers.Dense(128/2, activation='relu')(x)
x = layers.Dense(64/2, activation='relu')(x)
y = layers.Dropout(0.3)(x)

# Se introduce el output de la última capa convolucional como input junto con las features calculadas
x = layers.Dense(128/2, activation='relu')(features_input)
x = layers.Dense(64/2, activation='relu')(x)
x = layers.concatenate([y, x])
x = layers.Dense(64/2, activation='relu')(x)

output = layers.Dense(1, activation="sigmoid")(x)

model_features_neur05 = models.Model([img_input, features_input], output)

model_features_neur05.summary()

In [ ]:
checkpoint_filepath = DATA_DIR / 'model_features_neur05.{epoch:02d}-{val_loss:.4f}.hdf5'

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [ ]:
model_features_neur05.compile(optimizer='rmsprop',
              loss='mae',
              metrics=['mse'])

In [ ]:
history_model_features_neur05 = model_features_neur05.fit(x=[train_img4, train_features3], y=train_labels, epochs=50, validation_data=([test_img4, test_features3], test_labels), callbacks=model_checkpoint_callback)

In [ ]:
model_features_neur05.load_weights(DATA_DIR / 'model_features_neur05.48-0.1297.hdf5')

In [ ]:
model_features_neur05.save("model_features_neur05.h5")

In [ ]:
# model_features_neur2 = keras.models.load_model("model_features_neur2.h5")

In [ ]:
plt.plot(range(50), history_model_features_neur05.history['loss'],range(50),history_model_features_neur05.history['val_loss'])
plt.xlabel('Epochs')
plt.ylabel('MSE')

In [ ]:
predictions = model_features_neur05.predict([train_img4, train_features3])

In [ ]:
rounded_result, acc, rmse, rmae = reconversion_target(predictions, train_labels, train_subset)

In [ ]:
acc, rmse, rmae

In [ ]:
mae_percent_error(predictions, train_labels)

In [ ]:
predictions = model_features_neur05.predict([test_img4, test_features3])

In [ ]:
rounded_result, acc, rmse, rmae = reconversion_target(predictions, test_labels, test_subset)

In [ ]:
acc, rmse, rmae

In [ ]:
mae_percent_error(predictions, test_labels)

In [ ]:
evaluations = model_features_neur05.evaluate([train_img4, train_features3], train_labels)

In [ ]:
evaluations = model_features_neur05.evaluate([test_img4, test_features3], test_labels)

### Neuronas. x 4

In [ ]:
# Agregar capas de entrada
img_input = layers.Input(shape=(SHAPE[1], SHAPE[2], SHAPE[0]))
features_input = layers.Input(shape=(number_of_features,))

# Agregar capas convolucionales. Procesado imagen
x = layers.Conv2D(32*4, (3, 3), activation='relu')(img_input)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64*4, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64*4, (3, 3), activation='relu')(x)

# Aplanar los datos y agregar capas densas (fully connected)
x = layers.Flatten()(x)
x = layers.Dense(128*4, activation='relu')(x)
x = layers.Dense(64*4, activation='relu')(x)
y = layers.Dropout(0.3)(x)

# Se introduce el output de la última capa convolucional como input junto con las features calculadas
x = layers.Dense(128*4, activation='relu')(features_input)
x = layers.Dense(64*4, activation='relu')(x)
x = layers.concatenate([y, x])
x = layers.Dense(64*4, activation='relu')(x)

output = layers.Dense(1, activation="sigmoid")(x)

model_features_neur4 = models.Model([img_input, features_input], output)

model_features_neur4.summary()

In [ ]:
checkpoint_filepath = DATA_DIR / 'model_features_neur4.{epoch:02d}-{val_loss:.4f}.hdf5'

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [ ]:
model_features_neur4.compile(optimizer='rmsprop',
              loss='mse',
              metrics=['mae', tf.keras.metrics.mean_absolute_percentage_error])

In [ ]:
history_model_features_neur4 = model_features_neur4.fit(x=[train_img4, train_features3], y=train_labels, epochs=50, validation_data=([test_img4, test_features3], test_labels), callbacks=model_checkpoint_callback)

In [ ]:
model_features_neur4.load_weights(DATA_DIR / 'model_features_neur4.12-0.0334.hdf5')

In [ ]:
model_features_neur4.save("model_features_neur4.h5")

In [ ]:
# model_features_neur4 = keras.models.load_model("model_features_neur4.h5")

In [ ]:
plt.plot(range(50), history_model_features_neur4.history['loss'],range(50),history_model_features_neur4.history['val_loss'])
plt.xlabel('Epochs')
plt.ylabel('MSE')

In [ ]:
predictions = model_features_neur4.predict([train_img4, train_features3])

In [ ]:
rounded_result, acc, rmse, rmae = reconversion_target(predictions, train_labels, train_subset)

In [ ]:
acc, rmse, rmae

In [ ]:
mae_percent_error(predictions, train_labels)

In [ ]:
predictions = model_features_neur4.predict([test_img4, test_features3])

In [ ]:
rounded_result, acc, rmse, rmae = reconversion_target(predictions, test_labels, test_subset)

In [ ]:
acc, rmse, rmae

In [ ]:
mae_percent_error(predictions, test_labels)

In [ ]:
evaluations = model_features_neur4.evaluate([train_img4, train_features3], train_labels)

In [ ]:
evaluations = model_features_neur4.evaluate([test_img4, test_features3], test_labels)

### QI > 25

In [ ]:
# Agregar capas de entrada
img_input = layers.Input(shape=(SHAPE[1], SHAPE[2], SHAPE[0]))
features_input = layers.Input(shape=(number_of_features,))

# Agregar capas convolucionales. Procesado imagen
x = layers.Conv2D(32, (3, 3), activation='relu')(img_input)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)

# Aplanar los datos y agregar capas densas (fully connected)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)
y = layers.Dropout(0.3)(x)

# Se introduce el output de la última capa convolucional como input junto con las features calculadas
x = layers.Dense(128, activation='relu')(features_input)
x = layers.Dense(64, activation='relu')(x)
x = layers.concatenate([y, x])
x = layers.Dense(64, activation='relu')(x)

output = layers.Dense(1, activation="sigmoid")(x)

model_featuresqi25 = models.Model([img_input, features_input], output)

model_featuresqi25.summary()

In [ ]:
checkpoint_filepath = DATA_DIR / 'features_weightsqi25.{epoch:02d}-{val_loss:.4f}.hdf5'

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [ ]:
model_featuresqi25.compile(optimizer='rmsprop',
              loss='mae',
              metrics=['mse'])

In [ ]:
history_featuresqi25_model = model_featuresqi25.fit(x=[train_img4, train_features3], y=train_labels, epochs=100, validation_data=([test_img4, test_features3], test_labels), callbacks=model_checkpoint_callback)

In [ ]:
model_featuresqi25.load_weights(DATA_DIR / 'features_weightsqi25.90-0.1219.hdf5')

In [ ]:
model_featuresqi25.save("model_featuresqi25.h5")

In [ ]:
# model_features = keras.models.load_model("model_features.h5")

In [ ]:
flg,ax = plt.subplots()
ax.plot(range(100), history_featuresqi25_model.history['loss'], label='train')
ax.plot(range(100),history_featuresqi25_model.history['val_loss'], label='test')
ax.set_xlabel('Epochs')
ax.set_ylabel('MSE')
ax.legend()

In [ ]:
flg,ax = plt.subplots()
ax.plot(range(100), history_featuresqi25_model.history['mse'], label='train')
ax.plot(range(100),history_featuresqi25_model.history['val_mse'], label='test')
ax.set_xlabel('Epochs')
ax.set_ylabel('MSE')
ax.legend()

In [ ]:
predictions = model_featuresqi25.predict([train_img4, train_features3])

In [ ]:
rounded_result, acc, rmse, rmae = reconversion_target(predictions, train_labels, train_subset)

In [ ]:
acc, rmse, rmae

In [ ]:
mae_percent_error(predictions, train_labels)

In [ ]:
predictions = model_featuresqi25.predict([test_img4, test_features3])

In [ ]:
rounded_result, acc, rmse, rmae = reconversion_target(predictions, test_labels, test_subset)

In [ ]:
acc, rmse, rmae

In [ ]:
mae_percent_error(predictions, test_labels)

In [ ]:
evaluations = model_featuresqi25.evaluate([train_img4, train_features3], train_labels)

In [ ]:
evaluations = model_featuresqi25.evaluate([test_img4, test_features3], test_labels)

### QI > 50

In [ ]:
# Agregar capas de entrada
img_input = layers.Input(shape=(SHAPE[1], SHAPE[2], SHAPE[0]))
features_input = layers.Input(shape=(number_of_features,))

# Agregar capas convolucionales. Procesado imagen
x = layers.Conv2D(32, (3, 3), activation='relu')(img_input)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)

# Aplanar los datos y agregar capas densas (fully connected)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)
y = layers.Dropout(0.3)(x)

# Se introduce el output de la última capa convolucional como input junto con las features calculadas
x = layers.Dense(128, activation='relu')(features_input)
x = layers.Dense(64, activation='relu')(x)
x = layers.concatenate([y, x])
x = layers.Dense(64, activation='relu')(x)

output = layers.Dense(1, activation="sigmoid")(x)

model_featuresqi50 = models.Model([img_input, features_input], output)

model_featuresqi50.summary()

In [ ]:
checkpoint_filepath = DATA_DIR / 'features_weightsqi50.{epoch:02d}-{val_loss:.4f}.hdf5'

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [ ]:
model_featuresqi50.compile(optimizer='rmsprop',
              loss='mae',
              metrics=['mse'])

In [ ]:
history_featuresqi50_model = model_featuresqi50.fit(x=[train_img4, train_features3], y=train_labels, epochs=15, validation_data=([test_img4, test_features3], test_labels), callbacks=model_checkpoint_callback)

In [ ]:
model_featuresqi50.load_weights(DATA_DIR / 'features_weightsqi50.44-0.1186.hdf5')

In [ ]:
model_featuresqi50.save("model_featuresqi25.h5")

In [ ]:
# model_features = keras.models.load_model("model_features.h5")

In [ ]:
flg,ax = plt.subplots()
ax.plot(range(100), history_featuresqi50_model.history['loss'], label='train')
ax.plot(range(100),history_featuresqi50_model.history['val_loss'], label='test')
ax.set_xlabel('Epochs')
ax.set_ylabel('MSE')
ax.legend()

In [ ]:
flg,ax = plt.subplots()
ax.plot(range(100), history_featuresqi50_model.history['mse'], label='train')
ax.plot(range(100),history_featuresqi50_model.history['val_mse'], label='test')
ax.set_xlabel('Epochs')
ax.set_ylabel('MSE')
ax.legend()

In [ ]:
predictions = model_featuresqi50.predict([train_img4, train_features3])

In [ ]:
rounded_result, acc, rmse, rmae = reconversion_target(predictions, train_labels, train_subset)

In [ ]:
acc, rmse, rmae

In [ ]:
mae_percent_error(predictions, train_labels)

In [ ]:
predictions = model_featuresqi50.predict([test_img4, test_features3])

In [ ]:
rounded_result, acc, rmse, rmae = reconversion_target(predictions, test_labels, test_subset)

In [ ]:
acc, rmse, rmae

In [ ]:
mae_percent_error(predictions, test_labels)

In [ ]:
evaluations = model_featuresqi50.evaluate([train_img4, train_features3], train_labels)

In [ ]:
evaluations = model_featuresqi50.evaluate([test_img4, test_features3], test_labels)

## Referencias

[1] Keras. API docs: https://keras.io/api/

[2] François Chollet. Deep Learning with Python. Manning Publications Co. Shelter Island, NY 11964. ISBN 9781617294433